# Программный комплекс на удаленном устройстве
На удаленном устройстве в папке проекта должно находится два файла:

OpenShowVar.py – свободно распространяемый модуль, который позволяет использовать переменные и структуры данных управляемого манипулятора для чтения и записи. Более подробно про модуль можно почитать по ссылке: 
https://scispace.com/pdf/controlling-kuka-industrial-robots-flexible-communication-1n6mpcqxsm.pdf

Kuka.py – библиотека команд для управления манипулятором с удаленного устройства.

# Библиотека Kuka.py
•	kuka.set_base (base)

Параметры:

base – номер базы от 1 до 32, предварительно откалиброванной на манипуляторе.

Результат:

Обращение к Case 1 и выбор заданной базы как систему отсчета.


In [ ]:
    def set_base(self, base):
        time.sleep(0.05)
        Base=(self.robot.read(("BASE_DATA[" + str(base)+"]"), False).decode())
        self.robot.write("COM_FRAME", Base)
        self.robot.write("COM_CASEVAR", "1")
        while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
            continue 

•	kuka.set_tool (tool)

Параметры:

tool – номер базы от 1 до 32, предварительно откалиброванной на манипуляторе.

Результат:

Обращение к Case 2 и выбор заданного инструмента как систему отсчета.

In [ ]:
 def set_tool(self, tool):
        time.sleep(0.05)
        Tool=(self.robot.read(("TOOL_DATA[" + str(tool)+"]"), False).decode())
        self.robot.write("COM_FRAME", Tool)
        self.robot.write("COM_CASEVAR", "2")
        while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
            continue     

•	kuka. set_speed (value)

Параметры:

value – скорость перемещения манипулятора в % от 0 до 100.

Результат:

Обращение к Case 3 и задание скорости выполнения программы.

In [ ]:
def set_speed(self, value):
    time.sleep(0.05)
    self.robot.write("COM_VALUE1", str(value), False)
    self.robot.write("COM_CASEVAR", "3", False)
    while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
        continue     

•	kuka.lin_continuous (kuka, massiv)

Параметры:

kuka – подключенный к системе дистанционного управления манипулятор.

massiv – массив координат в формате numpy.

Результат:

Обращение к функции massiv_coord и формирование массива координат в формате, который может интерпретировать манипулятор. Полученный массив отправляется на манипулятор с помощью функции send_Frame_array. При этом в переменную COM_LENGTH записывается длина массива и происходит обращение к Case 4. Тип движения LIN.

•	kuka. send_Frame (arr, system_variable="")

Параметры:

arr – массив координат в формате numpy.

Результат:

Отправка в манипулятор значений точек движения в формате:

{FRAME: X значения, Y значения, Z значения, A значения, B значения, C значения}, 
где X, Y, Z – координаты относительно выбранной базы, A, B, C – углы поворота, A – угол, вращающийся вокруг оси X, B – угол, вращающийся вокруг оси Y, C – угол, вращающийся вокруг оси Z.

In [ ]:
def lin_continuous(self, kuka, massiv):
    time.sleep(0.05)
    arr=self.massiv_coord(kuka, massiv)
    self.send_Frame_array(arr)
    self.robot.write("COM_LENGTH", str(arr.shape[0]-1), False)
    self.robot.write("COM_CASEVAR", "4", False)
    while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
        continue
        
def massiv_coord(self, kuka, massiv):  
    kuka.read_cartesian()
    start_point = np.array([kuka.x_cartesian, kuka.y_cartesian, kuka.z_cartesian, kuka.A_cartesian, kuka.B_cartesian, kuka.C_cartesian])
    trajectory = np.vstack([start_point,massiv])
    return trajectory
           
def send_Frame_array(self, arr):
    for i in range(len(arr)):
        index_string = ("COM_FRAME_ARRAY[" + str(i) + "]")
        self.send_Frame(arr[i], index_string)
            
def send_Frame(self, arr, system_variable=""):
    string_arr = []
    for i in range(len(arr)):
        string_arr.append(str(arr[i]))
    cartesian_string = ("{FRAME: X " + string_arr[0] + ", Y " + string_arr[1] + ", Z "+ string_arr[2] + ", A " + string_arr[3] + ", B " + string_arr[4] + ", C " + string_arr[5] + "}")
    self.robot.write(system_variable, cartesian_string, False)
    

•	kuka.ptp_continuous (kuka, massiv)

Параметры:

kuka – подключенный к системе дистанционного управления манипулятор.

massiv – массив координат в формате numpy.

Результат:

Обращение к функции massiv_coord и формирование массива координат в формате, который может интерпретировать манипулятор. Полученный массив отправляется на манипулятор с помощью функции send_Frame_array. При этом в переменную COM_LENGTH записывается длина массива и происходит обращение к Case 5. Тип движения PTP.

In [ ]:
def ptp_continuous(self, kuka, massiv):
    time.sleep(0.05)
    arr=self.massiv_coord(kuka, massiv)
    self.send_Frame_array(arr)
    self.robot.write("COM_LENGTH", str(arr.shape[0]-1), False)
    self.robot.write("COM_CASEVAR", "5", False)
    while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
        continue  

•	kuka.open_grip()

Параметры:

-

Результат:

Управление цифровыми выходами OUT[1] и OUT[2], к которым подключен пневмораспределитель и пальцевый захват. Открытие захвата и обращение к Case 7.

•	kuka.close_grip()

Параметры:

-

Результат:

Управление цифровыми выходами OUT[1] и OUT[2], к которым подключен пневмораспределитель и пальцевый захват. Закрытие захвата и обращение к Case 7.

•	kuka.vacuum_on()

Параметры:

-

Результат:

Управление цифровым выходом OUT[3], к которому подключен вакуумный захват. Включение захвата и обращение к Case 8.

•	kuka.vacuum_off()

Параметры:

-

Результат:

Управление цифровым выходом OUT[3], к которому подключен вакуумный захват. Отключение захвата и обращение к Case 8.

In [ ]:
def open_grip(self):
    time.sleep(0.05)
    self.robot.write('OUT5', 'False')
    self.robot.write('OUT6', 'True')
    self.robot.write("COM_CASEVAR", "7", False)
    while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
        continue 

def close_grip(self):
    time.sleep(0.05)
    self.robot.write('OUT5', 'True')
    self.robot.write('OUT6', 'False')
    self.robot.write("COM_CASEVAR", "7", False)
    while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
        continue        

def vacuum_on(self):
    time.sleep(0.05)
    self.robot.write('OUT7', 'True')
    self.robot.write("COM_CASEVAR", "8", False)
    while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
        continue 

def vacuum_off(self):
    time.sleep(0.05)
    self.robot.write('OUT7', 'False')
    self.robot.write("COM_CASEVAR", "8", False)
    while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
        continue 

•	kuka. quit()

Параметры:

-

Результат:

Обращение к Case 9 и выход из программы Server.src.

In [ ]:
def quit(self):
    time.sleep(0.05)
    self.robot.write("COM_CASEVAR", "9", False)
    while int(self.robot.read("COM_CASEVAR", False).decode())!=0:
        continue  

•	kuka. read_base()

Параметры:

-

Результат:

Считывание значения системной переменной $BASE и запись её координат в переменные: base_frame_x, base_frame_y, base_frame_z, base_frame_A, base_frame_B, base_frame_C и общих координаты в переменную base_frame.

•	kuka. read_tool()

Параметры:

-

Результат:

Считывание значения системной переменной $TOOL и запись координаты инструмента в переменные: tool_frame_x, tool_frame_y, tool_frame_z, tool_frame_A, tool_frame_B, tool_frame_C и общих координат в переменную tool_frame.

•	kuka. read_joint()

Параметры:

-

Результат:

Считывание значения системной переменной $AXIS_ACT и запись значений углов сочленений шестиосевого манипулятора в переменные: A1_joint, A2_joint, A3_joint, A4_joint, A5_joint, A6_joint и общих значений углов сочленений в переменную global_position_joint

•	kuka. read_cartesian()

Параметры:

-

Результат:

Считывание значения системной переменной $POS_ACT и запись значений обобщеных координат в переменные: x_cartesian, y_cartesian, z_cartesian, A_cartesian, B_cartesian, C_cartesian и общих значений обобщеных координат в переменную global_position. В случае использования манипулятора на передвижной платформе используются еще 6 обобщеных координат: E1_cartesian, E2_cartesian, E3_cartesian, E4_cartesian, E5_cartesian, E6_cartesian.

In [ ]:
def read_base(self):
    string = self.robot.read("$BASE", False).decode()
    string = string.replace(',', '')
    string = string.replace('{', '')
    string = string.replace('}', '')
    string = string.split()
    self.base_frame_x = float(string[2])
    self.base_frame_y = float(string[4])
    self.base_frame_z = float(string[6])
    self.base_frame_A = float(string[8])
    self.base_frame_B = float(string[10])
    self.base_frame_C = float(string[12])
    self.base_frame = np.array([self.base_frame_x, self.base_frame_y, self.base_frame_z, self.base_frame_A, self.base_frame_B, self.base_frame_C])

def read_tool(self):
    string = self.robot.read("$TOOL", False).decode()
    string = string.replace(',', '')
    string = string.replace('{', '')
    string = string.replace('}', '')
    string = string.split()
    self.tool_frame_x = float(string[2])
    self.tool_frame_y = float(string[4])
    self.tool_frame_z = float(string[6])
    self.tool_frame_A = float(string[8])
    self.tool_frame_B = float(string[10])
    self.tool_frame_C = float(string[12])
    self.tool_frame = np.array([self.tool_frame_x, self.tool_frame_y, self.tool_frame_z, self.tool_frame_A, self.tool_frame_B, self.tool_frame_C])

def read_joint(self):
    joint_string = self.robot.read("$AXIS_ACT", False).decode()
    joint_string = joint_string.replace(',', '')
    joint = joint_string.split()
    self.A1_joint = float(joint[2])
    self.A2_joint = float(joint[4])
    self.A3_joint = float(joint[6])
    self.A4_joint = float(joint[8])
    self.A5_joint = float(joint[10])
    self.A6_joint = float(joint[12])
    self.global_position_joint = np.array([self.A1_joint, self.A2_joint, self.A3_joint, self.A4_joint, self.A5_joint, self.A6_joint])

def read_cartesian(self):
    cartesian_string = self.robot.read("$POS_ACT", False).decode()
    cartesian_string = cartesian_string.replace(',', '')
    cartesian = cartesian_string.split()
    self.x_cartesian = float(cartesian[2])
    self.y_cartesian = float(cartesian[4])
    self.z_cartesian = float(cartesian[6])
    self.A_cartesian = float(cartesian[8])
    self.B_cartesian = float(cartesian[10])
    self.C_cartesian = float(cartesian[12])
    self.E1_cartesian = float(cartesian[14])
    self.E2_cartesian = float(cartesian[16])
    self.E3_cartesian = float(cartesian[18])
    self.E4_cartesian = float(cartesian[20])
    self.E5_cartesian = float(cartesian[22])
    self.E6_cartesian = float(cartesian[24])
    self.global_position = np.array([self.x_cartesian, self.y_cartesian, self.z_cartesian, self.A_cartesian, self.B_cartesian, self.C_cartesian])

•	kuka. robot_read(types)

Параметры:

types – название переменной в формате “$Название переменной”.

Результат:

Чтение с манипулятора значения системной переменной, переданной в types и вывод результата на экран.

In [ ]:
def robot_read(self, types):  
    print(self.robot.read(str(types), False).decode())